<table>
    <tr>
        <td><img src="https://www.acofi.edu.co/eiei2016/wp-content/uploads/2016/09/Logo-Universidad-EIA.jpg" width="250"/></td>
        <td>&nbsp;</td>
        <td>
        <td><img src="https://raw.githubusercontent.com/Fabian830348/Bases_Datos/refs/heads/master/Logo_EICT_horizontal_ESPANOL%20(1).png" width="300"/></td>
        <td>&nbsp;</td>
        <td>
            <tp><p style="font-size:99%;text-align:center">Machine Learning_PLN </p></tp>
            <tp><p style="font-size:115%;text-align:center">Diplomado 2025-2</p></tp>
            <tp><p style="font-size:115%;text-align:center">Prof. Fabián Sánchez</p></tp>
        </td>
    </tr>
</table>

# <FONT SIZE=5 COLOR="purple"> Topic Modeling </FONT>

El ***Topic Modeling** (o modelado de temas) es una técnica fundamental del Procesamiento del Lenguaje Natural (PLN) que permite descubrir automáticamente los temas o tópicos latentes que aparecen en un conjunto grande de textos, sin necesidad de que estén etiquetados.

Vamos a cargar la librerías de trabajo

In [1]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [16]:
# librería de PLN en español
from nltk.corpus import stopwords
stopwords_sp = stopwords.words('spanish')

In [2]:
pip install gensim

In [3]:
!pip install numpy==1.24.3

  Using cached numpy-1.24.3.tar.gz (10.9 MB)
  Installing build dependencies ... done
  error: subprocess-exited-with-error
  
  × Getting requirements to build wheel did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Getting requirements to build wheel ... error
error: subprocess-exited-with-error

× Getting requirements to build wheel did not run successfully.
│ exit code: 1
╰─> See above for output.

note: This error originates from a subprocess, and is likely not a problem with pip.


In [4]:
# cargue las librerías necesarias para trabajar los puntos del trabajo

# librerías para procesar
#import numpy as np
import pandas as pd

# expresiones regulares
import re

# librerías para graficar
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

# Count vectorizer usando NLTK
from sklearn.feature_extraction.text import TfidfVectorizer

# Para el algoritmo k-means
from sklearn.cluster import KMeans

# Algunos elementos de preprocesamiento: escalamiento y selección de k en k-means
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_samples, silhouette_score

# Para los algotitmos de machine learning
from sklearn.decomposition import PCA
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import NearestNeighbors

from sklearn.metrics import pairwise_distances_argmin_min

#Para omitir los warnings
import warnings
warnings.filterwarnings("ignore")

In [43]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [5]:
# indique la función de limpieza de texto que va a usar.

def pre_procesado(texto):
    texto = texto.lower()
    texto = re.sub('á', 'a', texto)
    texto = re.sub('é', 'e', texto)
    texto = re.sub('í', 'i', texto)
    texto = re.sub('ó', 'o', texto)
    texto = re.sub('ú', 'u', texto)
    texto = re.sub("[^a-züéáíóúñ]", " ", texto)
  #  texto = re.sub(r"[\W\d_]+", " ", texto)
    texto = [palabra for palabra in texto.split() if (len(palabra)>3 and palabra not in stopwords_sp) | (palabra == "no")]
    texto = " ".join(texto)
    return texto

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# <FONT SIZE=5 COLOR="purple"> Ejemplo 1 </FONT>

In [33]:
ruta = "/content/drive/MyDrive/1.3_Diplomado_Salud_UR_EIA/hospital_comentarios2.xlsx"

In [35]:
datos = pd.read_excel(ruta)

In [36]:
datos.head()

,id,fecha,ciudad,canal,tema,comentario,sentimiento_label,rating_1a5
0,522,2024-11-21 00:20:03,Pereira,Correo electrónico,alimentación,"Sin embargo, en oftalmología me explicaron tod...",positivo,5
1,738,2025-01-30 08:45:03,Neiva,Google Reviews,telemedicina,Muy agradecido con el equipo de Telemedicina; ...,positivo,4
2,741,2025-05-30 06:59:03,Cartagena,Google Reviews,información y señalización,Atención promedio en Cardiología; información ...,neutro,3
3,661,2025-09-05 05:42:03,Medellín,Kiosko en sala de espera,alimentación,Volvería a Fisioterapia; alimentación y calide...,positivo,5
4,412,2025-07-06 20:10:03,Manizales,WhatsApp,autorizaciones EPS,"Buen trato en Dermatología, autorizaciones EPS...",positivo,5


In [39]:
textos = datos["comentario"]

In [40]:
# Haga una breve exploración de los datos
textos.head()

,comentario
0,"Sin embargo, en oftalmología me explicaron tod..."
1,Muy agradecido con el equipo de Telemedicina; ...
2,Atención promedio en Cardiología; información ...
3,Volvería a Fisioterapia; alimentación y calide...
4,"Buen trato en Dermatología, autorizaciones EPS..."


In [41]:
# ¿cuál es el tamaño de los datos?
textos.shape

(1000,)

In [42]:
# haga el proceso de limpieza y denomine la variable como documento
documentos = [pre_procesado(oracion)  for oracion in textos]
documentos[0:10]

['embargo oftalmologia explicaron claridad alimentacion problemas',
 'agradecido equipo telemedicina telemedicina impecable',
 'atencion promedio cardiologia informacion señalizacion bien destacar',
 'volveria fisioterapia alimentacion calidez humana increibles',
 'buen trato dermatologia autorizaciones bien manejado',
 'agradecido equipo cirugia telemedicina impecable',
 'resumen pesima experiencia ginecologia agendamiento citas dejo desear',
 'pesar atencion promedio nefrologia diagnostico bien destacar',
 'atencion promedio atencion cliente seguridad paciente bien destacar',
 'atencion promedio cardiologia medicacion bien destacar']

In [44]:
# haga el proceso de tokenización y llamelo : doc_tok
doc_tok = [word.split() for word in documentos]
doc_tok[0:10]

[['embargo',
  'oftalmologia',
  'explicaron',
  'claridad',
  'alimentacion',
  'problemas'],
 ['agradecido', 'equipo', 'telemedicina', 'telemedicina', 'impecable'],
 ['atencion',
  'promedio',
  'cardiologia',
  'informacion',
  'señalizacion',
  'bien',
  'destacar'],
 ['volveria',
  'fisioterapia',
  'alimentacion',
  'calidez',
  'humana',
  'increibles'],
 ['buen', 'trato', 'dermatologia', 'autorizaciones', 'bien', 'manejado'],
 ['agradecido', 'equipo', 'cirugia', 'telemedicina', 'impecable'],
 ['resumen',
  'pesima',
  'experiencia',
  'ginecologia',
  'agendamiento',
  'citas',
  'dejo',
  'desear'],
 ['pesar',
  'atencion',
  'promedio',
  'nefrologia',
  'diagnostico',
  'bien',
  'destacar'],
 ['atencion',
  'promedio',
  'atencion',
  'cliente',
  'seguridad',
  'paciente',
  'bien',
  'destacar'],
 ['atencion', 'promedio', 'cardiologia', 'medicacion', 'bien', 'destacar']]

In [45]:
# construya el diccionario y el corpus con Gensim

# diccionario de gensim
from gensim.corpora import Dictionary

# documents es la lista tokenizada de documentos
dictionary = Dictionary(doc_tok)
dictionary.filter_extremes(no_below=3,
                          no_above=0.5,
                          keep_tokens=None
                           )
# Creamos la Bolsa de Palabras
corpus = [dictionary.doc2bow(doc) for doc in doc_tok]

In [46]:
# genere el modelo en Gensim con 7-8 tópicos
from gensim.models import LdaModel
lda_model = LdaModel(corpus=corpus,               # corpus es la bolsa de palabras de Gensim
                     num_topics=8,                # número de temas que queremos
                     id2word=dictionary)

In [47]:
pip install pyLDAvis

In [48]:
import pyLDAvis
import pyLDAvis.gensim

In [49]:
vis = pyLDAvis.gensim.prepare(lda_model,           # modelo
                              corpus,              # corpus
                              dictionary,          # diccionario de gensim
                              R = 8)               # número de palabras claves
pyLDAvis.display(vis)

# <FONT SIZE=5 COLOR="purple"> Ejemplo 2 </FONT>

Hacer lo mismo con

In [ ]:
#ruta = "/content/drive/MyDrive/1.3_Diplomado_Salud_UR_EIA/comentarios_salud_500.csv"